In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import Series,DataFrame
import matplotlib
from decimal import Decimal
import sys
import urllib
import json
import datetime
import pylab as pl
import re
import os
import csv
import time
from datetime import datetime,timedelta
import statsmodels.api as sm
import re
from geopy.distance import geodesic
import math
import warnings
warnings.filterwarnings("ignore")

In [2]:
def my_geodesic(a,b,c,d):
    return geodesic((a,b),(c,d)).m

In [91]:
file_path='D:/dingdang/三方长时间不接起'

order = pd.read_csv(file_path+'/order.csv')

station_shop = pd.read_csv(file_path+'/station_ship.csv')

order_time_3 = pd.read_csv(file_path+'/order_time4.csv')

station_shop.rename(columns={'id':'station_id'},inplace=True)

In [92]:
order.sort_values('audit_time')

,order_id,order_status,deliver_type,delivery_id,shipping_type,confirm_time,finish_time,pharmacy_id,audit_time,expect_time,slow_arrive_time,order_lng,order_lat
47,59963635841365,7,1,2027918.0,1,2020-10-15 11:01:30,2020-10-15 11:21:27,100013,2020-10-15 11:00:00,NaN,NaN,116.382011,39.850637
323,59963635846863,7,1,2011954.0,1,2020-10-15 11:04:43,2020-10-15 11:22:27,200668,2020-10-15 11:00:00,NaN,NaN,116.537982,39.926600
332,59963632016888,7,1,2011954.0,1,2020-10-15 11:04:47,2020-10-15 11:30:30,200668,2020-10-15 11:00:01,NaN,NaN,116.546148,39.918445
2,59963632005288,7,1,2027464.0,1,2020-10-15 11:00:35,2020-10-15 11:09:39,205652,2020-10-15 11:00:01,NaN,NaN,114.088163,22.548092
541,59963600648988,7,8,0.0,1,2020-10-15 11:06:41,2020-10-15 11:49:49,201789,2020-10-15 11:00:01,NaN,NaN,114.273507,22.746724
4,59963617287213,7,8,0.0,1,2020-10-15 11:00:39,2020-10-15 11:29:33,202572,2020-10-15 11:00:01,NaN,NaN,114.107783,22.575335
20,59963512967264,7,3,0.0,1,2020-10-15 11:01:00,2020-10-15 11:42:50,202672,2020-10-15 11:00:02,NaN,NaN,116.340930,39.871891
22,59963591040682,7,1,2024817.0,1,2020-10-15 11:01:02,2020-10-15 11:10:11,200206,2020-10-15 11:00:02,NaN,NaN,116.306157,40.056658
21,59963594885243,7,1,2026936.0,1,2020-10-15 11:01:01,2020-10-15 11:19:45,204152,2020-10-15 11:00:02,NaN,NaN,113.936914,22.511602
19,59963601283288,7,1,2023651.0,1,2020-10-15 11:00:59,2020-10-15 11:32:35,202932,2020-10-15 11:00:02,NaN,NaN,116.364768,39.889470


In [5]:
def data_merge(order_time_3,order):
    order_time = pd.merge(order_time_3,order, how = 'inner', 
                on = 'order_id')
    ## 整理期望到达时间
    order_time['expect_time'] = order_time['expect_time'].fillna(0)
    order_time.loc[order_time['expect_time'] == 0,'expect_time'] = order_time['slow_arrive_time'].fillna(0)
    order_time['expect_time'] = pd.to_datetime(order_time['expect_time'])
    order_time['expect_time1'] = pd.to_datetime(order_time.loc[order_time['shipping_type'
            ] == 1,'audit_time'])+timedelta(minutes=38)
    order_time['expect_time'] = pd.to_datetime(order_time['expect_time'])
    order_time.loc[order_time['expect_time'] == '1970-01-01 00:00:00','expect_time'] = pd.to_datetime(order_time[
    'expect_time1'])

    order_time = order_time.drop(['slow_arrive_time','expect_time1'], axis = 1)
    order_time.loc[order_time['shipping_type'] == 4,'expect_time'] = pd.to_datetime(order_time['expect_time'])+timedelta(hours=1)
    return order_time
    

In [15]:
## 标记接单（<=1）改派及取货改派（<=1）的现象
def data_concat2(order_time,station_shop):
    receive_rate = order_time[['order_id','operate_type','status','operate_time']]
    receive_rate2 = pd.DataFrame()
    for order_id in receive_rate[(receive_rate['operate_type'] == 1)|
                            (receive_rate['operate_type'] == 4)].order_id.unique():
        order_num = receive_rate[receive_rate['order_id'] == order_id].sort_values(
            'operate_time')
        operate_time = order_num[(order_num['operate_type'] == 1)|
                            (order_num['operate_type'] == 4)][['operate_time']
                             ].sort_values('operate_time')
    # 判断谁送的单
        order_num['operate_num6'] = 0
    
    # 判断是否接单改派
        order_num['receive_num'] = 0

#     order_num['take_rate2'] = 0
    # 判断是否收货改派
        order_num['take_num'] = len(order_num[order_num['operate_type'] == 7])
        operate_time.index = range(len(operate_time))
        if len(operate_time) <= 1:
            order_num2 = order_num
            if len(order_num2[order_num2['operate_type'] == 6]) > 1:
                order_num2.loc[(order_num2['operate_time'] < order_num2[order_num2[
                    'operate_type'] == 6]['operate_time'].max())&(
                     order_num2['operate_type'] == 6),'operate_num6'] = 1
            order_num2['receive_num'] = len(order_num2[order_num2['operate_type'] == 6])

        # 用于多次手动发单
            order_num2['operate_num'] = 0
            receive_rate2 = pd.concat([order_num2,receive_rate2], 
                         axis=0)
        else:
            for i in range(len(operate_time)):
#             print(i,len(operate_time),operate_time)
                if i < len(operate_time)-1:
                    order_num2 = order_num[(order_num['operate_time'] >= operate_time.loc[i,'operate_time'])&
                        (order_num['operate_time'] < operate_time.loc[i+1,'operate_time']) ]
                    if len(order_num2[order_num2['operate_type'] == 6]) > 1:
                        order_num2.loc[(order_num2['operate_time'] < order_num2[order_num2[
                    'operate_type'] == 6]['operate_time'].max())&(
                    order_num2['operate_type'] == 6),'operate_num6'] = 1
                    order_num2['receive_num'] = len(order_num2[order_num2['operate_type'] == 6])

                    order_num2['operate_num'] = i
                else:
                    order_num2 = order_num[(order_num['operate_time'] >= operate_time.loc[
                        i,'operate_time'])]
                    if len(order_num2[order_num2['operate_type'] == 6]) > 1:
                        order_num2.loc[(order_num2['operate_time'] < order_num2[order_num2[
                         'operate_type'] == 6]['operate_time'].max())&(
                    order_num2['operate_type'] == 6),'operate_num6'] = 1
                    order_num2['receive_num'] = len(order_num2[order_num2['operate_type'] == 6])
                    order_num2['operate_num'] = i
                receive_rate2 = pd.concat([order_num2,receive_rate2], 
                         axis=0)
        
        
    order_time = pd.merge(order_time, receive_rate2, how = 'left', 
            on = ['order_id','operate_type','status','operate_time'])
#     print(order_time.head())
    
    ## 整理重试创建时间、接单时间、取货时间、超时取消时间
#     order_time_2 = order_time[(order_time['operate_type'] == 2)][['order_id','operate_time',
#                                                    'operate_num']]
#     order_time_2.rename(columns={'operate_time':'againoperate_time'},inplace=True)

    # 接单时间
    order_time_6 = order_time[(order_time['operate_type'] == 6)][['order_id','operate_time',
                                                   'operate_num']]
    order_time_6.rename(columns={'operate_time':'receiving_time'},inplace=True)

    # 取货时间
    order_time_7 = order_time[(order_time['operate_type'] == 7)][['order_id','operate_time',
                                                   'operate_num','operate_num6']]
    order_time_7.rename(columns={'operate_time':'take_time'},inplace=True)

    # 超时取消时间
#     order_time_3 = order_time[(order_time['operate_type'] == 3)][['order_id','operate_time',
#                                                    'operate_num','operate_num6']]
#     order_time_3.rename(columns={'operate_time':'out_time'},inplace=True)

    order_time2 = pd.merge(pd.merge(order_time[(order_time['operate_type'
                ] == 1)|(order_time['operate_type'] == 4)],order_time_6[['order_id','receiving_time']],how = 'left', 
                on = ['order_id']),order_time_7[['order_id','take_time']],how = 'left', 
                on = ['order_id'])
    order_all = pd.merge(order_time2,station_shop, how = 'inner', 
                on = 'pharmacy_id').drop_duplicates()
    
    return order_all

In [51]:
def order_operate2_data(order_all):
    a = order_all[['operate_num','order_id']].groupby('order_id').max()
    a.rename(columns={'operate_num':'all_num'},inplace=True)

    order_all = pd.merge(order_all,a, how = 'inner', 
                on = 'order_id')
    order_all2 = order_all
    order_all2['operate_time2'] = np.nan
    order_all2['delivery_type2'] = np.nan
    
    order_operate2 = pd.DataFrame()
    for order_id in order_all2[order_all2['all_num'] >= 1].order_id.unique():
        order_operate = order_all2[order_all2['order_id'] == order_id]
        for i in range(0,order_operate.operate_num.max().astype('int')):
            order_operate.loc[order_operate['operate_num'] == i,'operate_time2'
               ] = order_operate.loc[order_operate['operate_num'] == i+1,'operate_time'
                         ][order_operate[order_operate['operate_num'] == i+1].index[0]]
        
            order_operate.loc[order_operate['operate_num'] == i,'delivery_type2'
               ] = order_operate.loc[order_operate['operate_num'] == i+1,'delivery_type'
                         ][order_operate[order_operate['operate_num'] == i+1].index[0]]
        
        order_operate2 = pd.concat([order_operate2,order_operate]
                               ,axis=0).drop_duplicates()
    order_operate2 = pd.concat([order_operate2,order_all2[order_all2['all_num'] < 1]]
                               ,axis=0).drop_duplicates()
    return order_operate2

In [53]:
def order_operate3_data(order_operate2):
    order_operate2['diff_time'] = (pd.to_datetime(order_operate2['operate_time2']
           )-pd.to_datetime(order_operate2['operate_time'])).dt.total_seconds()
    order_operate3 = order_operate2
    order_operate3 = order_operate3[(order_operate3['receive_num']<2)&
                           (order_operate3['take_num']<2)]
    ## 接起率
    order_operate3['receive_rate'] = 0
    order_operate3.loc[order_operate3['receiving_time'].isna()== False,'receive_rate'] = 100
    order_operate3['take_rate'] = 0
    order_operate3.loc[order_operate3['take_time'].isna()== False,'take_rate'] = 100
    ## 加时间
    order_operate3['hour'] = pd.to_datetime(order_operate3['operate_time']).dt.hour
    ## 加距离
    order_operate3['dist'] = order_operate3[['station_lat','station_lng','order_lat',
        'order_lng']].dropna().apply(
        lambda row:my_geodesic(row['station_lat'], row['station_lng'],
        row['order_lat'], row['order_lng']), axis = 1)*1.4 # 遍历行
    
    ## 接单时长、取货时长、剩余时长
    order_operate3['receive_diff_time'] = round((pd.to_datetime(order_operate3[
        'receiving_time'])-pd.to_datetime(order_operate3['operate_time'])
                     )/ np.timedelta64(1, 's'),2)
    order_operate3['take_diff_time'] = round((pd.to_datetime(order_operate3['take_time']
               )-pd.to_datetime(order_operate3['receiving_time'])
                     )/ np.timedelta64(1, 's'),2)
    order_operate3['remnant_diff_time'] = round((pd.to_datetime(order_operate3['expect_time']
               )-pd.to_datetime(order_operate3['operate_time'])
                     )/ np.timedelta64(1, 's'),2)
    order_operate3['takeall_diff_time'] = round((order_operate3['receive_diff_time'
                ]+order_operate3['take_diff_time']),2)
    order_operate3['wait_diff_time'] = round((pd.to_datetime(order_operate3['operate_time']
               )-pd.to_datetime(order_operate3['audit_time'])
                     )/ np.timedelta64(1, 's'),2)
    order_operate3['interval_time'] = round((pd.to_datetime(order_operate3['expect_time']
               )-pd.to_datetime(order_operate3['audit_time'])
                     )/ np.timedelta64(1, 's'),2)
    
    ## 次数
    operate_num = order_operate3[['city_name','order_id']].groupby('order_id').count(
               ).reset_index()
    operate_num.rename(columns={'city_id':'frequency_num'},inplace=True)
    order_all = pd.merge(order_operate3, operate_num, how = 'left', 
                on = 'order_id')

## 取消率
    order_operate3['cance_rate'] = 0
    order_operate3.loc[(order_operate3['order_status'] == 10)|
                   (order_operate3['order_status'] == 8),'cance_rate'] = 100

## 是否超时
    order_operate3['arrive_time'] = order_operate3.loc[order_operate3['order_status'] == 7,
                            'finish_time']
    order_operate3['intime'] = round((pd.to_datetime(order_operate3['expect_time']
                )-pd.to_datetime(order_operate3['arrive_time'])
                     )/ np.timedelta64(1, 's'),2)
    order_operate3['intime'] = pd.cut(order_operate3['intime'],
                                  [order_operate3['intime'].min()-1,0,
                            order_operate3['intime'].max()],labels=[0,100]).astype(int)
    order_operate3.loc[order_operate3['intime']<0,'intime'] = np.nan
    return order_operate3
    
    
    

In [100]:
def order_operate_sy_data(order_operate3):
    order_2operate = order_operate3[(order_operate3['all_num'] >= 1)]
    order_2operate['diff_deliver'] = order_2operate['delivery_type']-order_2operate[
                               'delivery_type2']
    order_operate_sy = order_2operate[(order_2operate['diff_time'] > 7*60)&
               (order_2operate['diff_time'] < 8*60)&
            (order_2operate['diff_deliver'] != 0)&
             (order_2operate['shipping_type'] == 1)]
    return order_operate_sy

In [44]:
def order_operate_dz_data(order_operate3):
    order_operate_dz = order_operate3[((order_operate3['receive_diff_time'] > 7.5*60)|
                    (order_operate3['receive_rate'] == 0))&
                    (order_operate3['shipping_type'] == 1)]
    return order_operate_dz

In [112]:
def data3_data(order_operate_sy,order_operate3,type):
    data3 = pd.DataFrame(order_operate_sy[['receive_rate',
        'cance_rate','intime','order_id']].drop_duplicates().agg(
        {'receive_rate':np.mean,'cance_rate':np.mean,
        'intime':np.mean,'order_id':len})).T
    data3['rate'] = round(len(order_operate_sy.order_id.unique())/len(order_operate3[
              (order_operate3['shipping_type'] == 1)].order_id.unique())*100,2)
    data3.index = type
    return data3

In [93]:
order_time = data_merge(order_time_3,order)
order_all = data_concat2(order_time,station_shop)
order_operate2 = order_operate2_data(order_all)
order_operate3 = order_operate3_data(order_operate2)

In [97]:
# order_operate3['type'] = order_operate3['order_id'].astype('str')
# order_operate3['type'] = order_operate3['type'].dropna().apply(lambda x: re.sub('[0-4]$',
#                 '实验组', x)).apply(lambda x: re.sub('[5-9]$',
#                 '对照组', x)).apply(lambda x: re.sub('[0-9]',
#                 '', x))

In [101]:
order_operate_sy = order_operate_sy_data(order_operate3[(order_operate3['type'
                                                            ] == '实验组')])
order_operate_dz = order_operate_dz_data(order_operate3[(order_operate3['type'
                                                                       ] == '对照组')])

In [113]:
pd.concat([data3_data(order_operate_sy,order_operate3[(order_operate3['type'
        ] == '实验组')],['实验组']),
        data3_data(order_operate_dz,order_operate3[(order_operate3['type'
        ] == '对照组')],['对照组'])], axis=0)

,receive_rate,cance_rate,intime,order_id,rate
实验组,71.844660,4.854369,15.957447,103.0,7.32
对照组,53.633218,9.342561,21.912351,289.0,11.26
